## Object Relational Tutorial (1.x API)

The `SQLAlchemy Object Relational Mapper` presents a method of _associating_ __user-defined `Python classes` with `database tables`__, and _instances_ of those classes (_objects_) with `rows` in their corresponding `tables`. It _includes_ a system that __transparently synchronizes all changes in state__ between _objects_ and their _related rows_, called a `unit of work`, as well as a system for _expressing database queries_ in terms of the `user defined classes` and their `defined relationships` between each other.

The `ORM` is in _contrast_ to the `SQLAlchemy Expression Language`, upon which the `ORM` is constructed. Whereas the `SQL Expression Language`, introduced in `SQL Expression Language Tutorial (1.x API)`, presents a system of _representing_ the __primitive constructs of the relational database directly__ _without opinion_, the `ORM` presents a __high level__ and __abstracted__ `pattern of usage`, which itself is an example of applied usage of the _Expression Language_.

While there is __overlap__ among the `usage patterns` of the `ORM` and the `Expression Language`, the __similarities are more superficial__ than they may at first appear. One approaches the _structure and content of data_ from the `perspective` of a __`user-defined domain model`__ which is __transparently persisted and refreshed__ from its _underlying storage model_. The other approaches it from the perspective of __`literal schema` and `SQL expression` representations__ which are __explicitly composed into messages__ _consumed individually by the database_.

A _successful_ `application` may be constructed using the `Object Relational Mapper` __exclusively__. In _advanced situations_, an application constructed with the ORM may make _occasional usage_ of the `Expression Language` __directly__ in certain areas where __specific database interactions are required__.

#### Version Check

A _quick check_ to __verify__ that we are on at `least version 1.4` of `SQLAlchemy`.

In [1]:
import sqlalchemy

In [2]:
sqlalchemy.__version__

'1.4.41'